### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [42]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=10):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = ratings_mat.shape[0] * ratings_mat.shape[1] # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for it in range(iters):
    # update our sse
        old_sse = sse_accum
        sse_accum = 0

        # For each user-movie pair
        for u in range(n_users):
            for m in range(n_movies):
                
                # if the rating exists
                if ratings_mat[u, m] > 0:
                # compute the error as the actual minus the dot product of the user and movie latent features
                    cur_rating = ratings_mat[u, m]
                    u_vec = user_mat[u, :]
                    v_vec = movie_mat[:, m]
                    est_rating = np.dot(u_vec, v_vec)
                    err = cur_rating - est_rating
                    
#                     print(err)
                # Keep track of the total sum of squared errors for the matrix
                    sse_accum += err**2
                # update the values in each matrix in the direction of the gradient
#                     u_new = u_old
#                     v_new = v_old
                    for k in range(latent_features):
                        user_mat[u, k] += 2 * learning_rate * err * movie_mat[k, m]
                        movie_mat[k, m] += 2 * learning_rate * err * user_mat[u, k]
        
#         user_mat[u, :] = u_vec
#         movie_mat[:, m] = v_vec
        # print results
        print("%d \t\t %f" % (it+1, sse_accum / num_ratings))
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [27]:
ratings_mat

matrix([[10., 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [43]:
user_mat, movie_mat = FunkSVD(ratings_mat) # use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 57.921274
2 		 57.435089
3 		 56.943342
4 		 56.445984
5 		 55.942974
6 		 55.434281
7 		 54.919886
8 		 54.399780
9 		 53.873964
10 		 53.342452


In [44]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[0.61475553 1.11606392 0.88960344 0.91780397]
 [0.91342102 1.79865112 1.07423634 1.52725255]
 [1.0076406  1.61128681 1.18719487 1.88571583]
 [1.18416862 2.51868918 1.64480199 2.34795045]
 [0.82148988 1.50226317 0.99437796 1.50814137]
 [0.62841522 1.05515306 0.74249449 0.97549767]
 [0.74442059 1.60490773 1.16002349 1.41764081]
 [0.30205036 0.59884882 0.37252981 0.42786178]
 [1.39152907 2.29943744 1.65277626 2.39817975]
 [0.80903961 1.88793168 1.26451196 1.75530408]
 [0.70960193 1.26173814 0.92660639 1.08690155]
 [0.86547025 1.63473925 0.99670424 1.45531171]
 [0.99151452 1.74404038 1.31448475 1.97298102]
 [1.42891188 2.45630618 1.83482176 2.41071422]
 [1.37282029 2.62801176 1.90946585 2.26687079]
 [1.14868435 2.28846618 1.50431326 1.746971  ]
 [0.56002691 1.49614324 0.98546176 1.12587932]
 [0.68418457 1.48181308 1.04372352 1.27482436]
 [1.14411255 2.33354641 1.67087663 1.85574729]
 [0.79418487 1.60362065 1.05872713 1.55090742]]
[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [45]:
user_mat, movie_mat = FunkSVD(ratings_mat,latent_features=4, learning_rate=0.0005, iters=1000) #use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 61.207029
2 		 58.902644
3 		 56.460970
4 		 53.874124
5 		 51.140784
6 		 48.266871
7 		 45.266048
8 		 42.159899
9 		 38.977636
10 		 35.755231
11 		 32.533930
12 		 29.358181
13 		 26.273118
14 		 23.321831
15 		 20.542725
16 		 17.967257
17 		 15.618328
18 		 13.509485
19 		 11.644971
20 		 10.020542
21 		 8.624853
22 		 7.441186
23 		 6.449273
24 		 5.627015
25 		 4.951940
26 		 4.402348
27 		 3.958102
28 		 3.601116
29 		 3.315565
30 		 3.087912
31 		 2.906792
32 		 2.762809
33 		 2.648299
34 		 2.557079
35 		 2.484205
36 		 2.425755
37 		 2.378634
38 		 2.340413
39 		 2.309189
40 		 2.283478
41 		 2.262119
42 		 2.244210
43 		 2.229045
44 		 2.216071
45 		 2.204855
46 		 2.195058
47 		 2.186410
48 		 2.178699
49 		 2.171753
50 		 2.165437
51 		 2.159641
52 		 2.154276
53 		 2.149269
54 		 2.144560
55 		 2.140101
56 		 2.135850
57 		 2.131773
58 		 2.127843
59 		 2.124033
60 		 2.120325
61 		 2.116700
62 		 2.113144
63

627 		 0.006258
628 		 0.006178
629 		 0.006100
630 		 0.006023
631 		 0.005947
632 		 0.005872
633 		 0.005798
634 		 0.005724
635 		 0.005652
636 		 0.005581
637 		 0.005511
638 		 0.005442
639 		 0.005373
640 		 0.005306
641 		 0.005239
642 		 0.005174
643 		 0.005109
644 		 0.005045
645 		 0.004982
646 		 0.004920
647 		 0.004858
648 		 0.004798
649 		 0.004738
650 		 0.004679
651 		 0.004621
652 		 0.004563
653 		 0.004506
654 		 0.004450
655 		 0.004395
656 		 0.004341
657 		 0.004287
658 		 0.004234
659 		 0.004182
660 		 0.004130
661 		 0.004079
662 		 0.004029
663 		 0.003979
664 		 0.003930
665 		 0.003881
666 		 0.003834
667 		 0.003787
668 		 0.003740
669 		 0.003694
670 		 0.003649
671 		 0.003604
672 		 0.003560
673 		 0.003516
674 		 0.003473
675 		 0.003431
676 		 0.003389
677 		 0.003347
678 		 0.003307
679 		 0.003266
680 		 0.003226
681 		 0.003187
682 		 0.003148
683 		 0.003110
684 		 0.003072
685 		 0.003035
686 		 0.002998
687 		 0.002962
688 		 0.002926
689 		 0

In [46]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.98979587  9.99949914 10.00109639 10.0090594 ]
 [10.00275392  4.01299301  8.99768454  9.98914053]
 [ 7.98734217  8.99419016 10.00178571  5.01503793]
 [ 9.01462352  7.98984099 10.00095111  9.99329059]
 [ 9.98479377  4.99315969  9.00221771  9.01788156]
 [ 5.99925762  3.99883227 10.00014475  6.0015227 ]
 [ 9.00732827  8.00580633  9.99868609  8.98958856]
 [ 9.97843849  4.99269934  9.00287239  8.02367668]
 [ 7.00521671  8.00651409  9.99851015  7.99122308]
 [ 9.01321735  4.99988974  8.99874389  6.9887189 ]
 [ 8.98517856  8.01204936  9.9993765   8.00501563]
 [ 8.99279779  9.98936349 10.00214046  9.01339308]
 [10.00310021  9.00837702  9.99872263  7.99150437]
 [ 4.99458513  8.0053863   4.9993069   8.00152106]
 [10.01065793  7.99328739 10.00036377  9.99489254]
 [ 9.00567182  8.99109324 10.00096995 10.00084233]
 [ 8.9996399   8.01853113  7.99707204  7.98823418]
 [10.00357485  7.99629347  1.00016832  9.99942995]
 [ 4.99322971  6.00078157 10.00039777 10.0054547 ]
 [ 8.02541172  6.99432176  9.99

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [47]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [48]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat,latent_features=4, learning_rate=0.0005, iters=250)#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 54.318820
2 		 51.499003
3 		 48.572425
4 		 45.547592
5 		 42.441004
6 		 39.276755
7 		 36.085664
8 		 32.903862
9 		 29.770887
10 		 26.727379
11 		 23.812585
12 		 21.061918
13 		 18.504835
14 		 16.163275
15 		 14.050801
16 		 12.172505
17 		 10.525609
18 		 9.100608
19 		 7.882769
20 		 6.853752
21 		 5.993177
22 		 5.280010
23 		 4.693657
24 		 4.214769
25 		 3.825755
26 		 3.511051
27 		 3.257192
28 		 3.052748
29 		 2.888167
30 		 2.755576
31 		 2.648553
32 		 2.561912
33 		 2.491490
34 		 2.433971
35 		 2.386721
36 		 2.347654
37 		 2.315125
38 		 2.287835
39 		 2.264757
40 		 2.245080
41 		 2.228163
42 		 2.213496
43 		 2.200672
44 		 2.189367
45 		 2.179317
46 		 2.170313
47 		 2.162182
48 		 2.154783
49 		 2.148002
50 		 2.141742
51 		 2.135925
52 		 2.130485
53 		 2.125364
54 		 2.120517
55 		 2.115903
56 		 2.111488
57 		 2.107243
58 		 2.103143
59 		 2.099167
60 		 2.095295
61 		 2.091512
62 		 2.087804
63 		

In [49]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 9.80401831219638:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [51]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users,latent_features=4, learning_rate=0.0005, iters=20)#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 0.014425
2 		 0.012176
3 		 0.010252
4 		 0.008716
5 		 0.007542
6 		 0.006655
7 		 0.005974
8 		 0.005437
9 		 0.005003
10 		 0.004644
11 		 0.004342
12 		 0.004083
13 		 0.003858
14 		 0.003660
15 		 0.003485
16 		 0.003328
17 		 0.003186
18 		 0.003057
19 		 0.002940
20 		 0.002832


`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [57]:
# Replace each of the comments below with the correct values
num_ratings = np.sum(np.sum(~np.isnan(first_1000_users)))  # How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = first_1000_users.shape[0] * first_1000_users.shape[1] - num_ratings# How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [58]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
